<h1>Capstone Project - The Battle of Neighborhoods</h1>

## Introduction: Business Problem

The aim of this project is to find a safe and secure location for opening of a Japanese restaurant in Toronto, Canada. 

The first task would be to **choose the safest borough** by analysing crime data for opening a restaurant and **short listing a neighborhood**, where Japanese restaurants are not amongst the most commom venues, and yet **as close to the city as possible**.

Also, our Japanese restaurant target customers will have a **Average to above average median net household incomes**.

I will focus on the safest borough and explore its neighborhoods and the 10 most common venues in each neighborhood first, and then filtering the data with the Toronto neighborhood profiles.

## Data Needed

Following data sources will be needed to extract/generate the required information:

1. Toronto crime data from 2014 - 2019. 
https://www.kaggle.com/kapastor/toronto-police-data-crime-rates-by-neighbourhood

2.  City of Toronto Neighborhood Profiles for providing an overview of the neighborhoods in Toronto.  
https://www.toronto.ca/city-government/data-research-maps/neighbourhoods-communities/neighbourhood-profiles/

3. City of Toronto Open Data Catalogue. 
https://open.toronto.ca/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

4. Foursquare API to collect information on other venues/competitors in the neighborhoods of Toronto.


I will combine the necessary information to create a new data frame. And using the Four Square API to explore the neighborhood venues and to apply a machine-learning algorithm (k-means) to cluster the neighborhoods and present the findings by plotting it on maps using Folium.